In [1]:
import csv
import os
import cv2
import numpy as np
from PIL import ImageFile
from keras_preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import backend as K
from keras.applications import InceptionV3
from keras.models import Model
from keras import Sequential
from keras.activations import softmax
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
 

def process_csv(csv_path):
    frame_dict = {}
    class_list = []
    with open(csv_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headers = next(csv_reader)  # Records the first row of csv

        for row in csv_reader:
            if row[4] != "" and row[4] != 'isTrackView':
                frame_dict[row[1]] = row[4]
                class_list.append(row[4])

    csv_file.close()
    return frame_dict, class_list

Using TensorFlow backend.


In [3]:
frame_dict, class_list = process_csv("../../labels/Frame Tagger - Singapore.csv")
print(np.unique(class_list))

class_weights = class_weight.compute_class_weight('balanced', np.unique(class_list), class_list)

['isDriverView' 'isOtherView' 'isPitView' 'isSpectatorView']


In [2]:
#sess = tf.Session()
#tf.enable_eager_execution() 

def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

def weighted_accuracy(weights):
    #weights = K.variable(weights)
    
    def accuracy(y_true, y_pred):
        resIndices = K.argmax(y_pred, 1)
        testIndices = K.argmax(y_true, 1)
        
        # results = np.array([np.insert(np.zeros(4), index, 1) for index in resIndices])

        print(resIndices.shape)
        print(testIndices.shape)
        
        conf_matrix = confusion_matrix(resIndices, testIndices)
        
        true_pos = np.diag(conf_matrix)
        p = np.sum(conf_matrix, axis=0)
        r = np.sum(conf_matrix, axis=1)
        precision = [np.sum(true_pos[i] / p[i]) if p[i] > 0 else 0 for i in range(5)]
        recall = [np.sum(true_pos[i] / r[i]) if r[i] > 0 else 0 for i in range(5)]
        accuracy = np.sum(true_pos)/np.sum(conf_matrix)
        
        if print_results:
            print('Accuracy: {:.2%}'.format(accuracy))
            print('Precision: {:.2%}'.format(np.mean(precision)))
            print('Recall: {:.2%}'.format(np.mean(recall)))
            print('Confusion Matrix:')
            print(conf_matrix)
        return K.constant(accuracy)#, precision, recall
        
        #with sess.as_default():
        # y_true = y_true.numpy()
        # y_pred = y_pred.numpy()
        # weight = weights
        # weight = weight / K.sum(weight, axis=-1, keepdims=True)
        # correct = np.ones_like(y_true)
        # correct[np.arange(len(y_true)), y_pred.argmax(axis=-1)] = 0
        # correct = np.sum(np.abs(y_true - correct) * weight, axis=-1)
        # accuracy = np.sum(correct) / len(y_true)
        # weight = weights
        # weight = weight / K.sum(weight, axis=-1, keepdims=True)
        # correct = K.ones_like(y_true)
        # #correct[K.arange(K.int_shape(y_true)[0]), y_pred.argmax(axis=-1)] = 0
        # correct = K.sum(K.abs(y_true - correct) * weight, axis=-1, keepdims=True)
        # accuracy = K.mean(correct)
        #return K.constant(accuracy)
    return accuracy
#print(class_weights)

In [4]:
datagen = ImageDataGenerator()
train_it = datagen.flow_from_directory("../../data_other/train", target_size=(299, 299),
                                       batch_size=64, class_mode="categorical", color_mode="rgb")

test_it = datagen.flow_from_directory("../../data_other/test", target_size=(299, 299),
                                      batch_size=64, class_mode="categorical", color_mode="rgb")

Found 529 images belonging to 4 classes.
Found 25 images belonging to 4 classes.


In [22]:
original_model = InceptionV3()
bottleneck_input = original_model.get_layer(index=0).input
bottleneck_output = original_model.get_layer(index=-2).output
bottleneck_model = Model(inputs=bottleneck_input, outputs=bottleneck_output)

for layer in bottleneck_model.layers:
    layer.trainable = False

new_model = Sequential()
new_model.add(bottleneck_model)
new_model.add(Dense(4, activation=softmax, input_dim=2048))

In [23]:
# For a binary classification problem
new_model.compile(optimizer='adam',
                  loss='categorical_crossentropy', #weighted_categorical_crossentropy(class_weights),
                  metrics=['accuracy'])#, weighted_accuracy(class_weights)])

ImageFile.LOAD_TRUNCATED_IMAGES = True
new_model.fit_generator(train_it, class_weight=class_weights, epochs=10)

print(np.unique(class_list))

Epoch 1/10
9/9 [==============================] - 48s 5s/step - loss: 0.6602 - accuracy: 0.8034
Epoch 2/10
9/9 [==============================] - 45s 5s/step - loss: 0.5022 - accuracy: 0.8639
Epoch 3/10
9/9 [==============================] - 46s 5s/step - loss: 0.4251 - accuracy: 0.8790
Epoch 4/10
9/9 [==============================] - 46s 5s/step - loss: 0.3174 - accuracy: 0.8658
Epoch 5/10
9/9 [==============================] - 44s 5s/step - loss: 0.2590 - accuracy: 0.9055
Epoch 6/10
9/9 [==============================] - 45s 5s/step - loss: 0.2063 - accuracy: 0.9263
Epoch 7/10
9/9 [==============================] - 45s 5s/step - loss: 0.2004 - accuracy: 0.9338
Epoch 8/10
9/9 [==============================] - 45s 5s/step - loss: 0.1604 - accuracy: 0.9509
Epoch 9/10
9/9 [==============================] - 50s 6s/step - loss: 0.1410 - accuracy: 0.9622
Epoch 10/10
9/9 [==============================] - 51s 6s/step - loss: 0.1288 - accuracy: 0.9754
['isDriverView' 'isOtherView' 'isPitVie

In [20]:
from sklearn.metrics import confusion_matrix, classification_report

print(new_model.metrics_names)
a = new_model.evaluate_generator(test_it)
print(a)

['loss', 'accuracy']
[39.93602752685547, 0.4000000059604645]


In [24]:
#print(test_it.samples)
#print(train_it.samples)

#Confution Matrix and Classification Report
Y_pred = new_model.predict_generator(test_it, test_it.samples // 64+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_it.classes, y_pred))
print('Classification Report')
target_names = np.unique(class_list)
print(classification_report(test_it.classes, y_pred, target_names=target_names))

Y_pred = new_model.predict_generator(train_it, train_it.samples // 64+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix _ Train')
print(confusion_matrix(train_it.classes, y_pred))

Confusion Matrix
[[ 0  0  5  0]
 [ 1  0  5  0]
 [ 0  0 12  0]
 [ 0  0  2  0]]
Classification Report
                 precision    recall  f1-score   support

   isDriverView       0.00      0.00      0.00         5
    isOtherView       0.00      0.00      0.00         6
      isPitView       0.50      1.00      0.67        12
isSpectatorView       0.00      0.00      0.00         2

       accuracy                           0.48        25
      macro avg       0.12      0.25      0.17        25
   weighted avg       0.24      0.48      0.32        25

Confusion Matrix _ Train
[[ 12   0 445   0]
 [  0   0  22   0]
 [  3   0  41   0]
 [  0   0   6   0]]


In [25]:
new_model.save("OtherViewModel.h5")